In [43]:
import facebook
import pandas as pd
import pymongo
import time
from pymongo import MongoClient
from transformers import pipeline

connection = MongoClient("mongodb://localhost:27017")
db = connection.IndeedJobPostDB
df = pd.DataFrame(list(db.Job_Posts.find()))
df = df.sort_values(by=['Current_Date'], ascending=False)
df = df.reset_index(drop=True)
df

,_id,JobID,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Details,Apply_link
0,64e06c60e874df75938eec0e,NY14072023FRI001,FOOD SERVER ASSISTANT (BUSSER),Full-time,Park Hyatt New York,$17.29 - $23.06 an hour,NaN,"New York, NY 10019 (Midtown area)",Just posted,23-07-14,The Food Server Assistant is responsible prima...,https://www.indeed.com//rc/clk?jk=923ac23a84cb...
1,64e06c60e874df75938eec6c,TX14072023FRI1005,Host/Hostess,Part-time,Gyu-Kaku Japanese BBQ,NaN,Weekends as needed+2,"Houston, TX 77070 (Willowbrook area)",Just posted,23-07-14,Helpful attitude and friendly demeanor. He or ...,https://www.indeed.com//rc/clk?jk=9a6e26a88780...
2,64e06c60e874df75938eec6e,TX14072023FRI1007,Host,Part-time,Denny's - Westheimer Road,NaN,Weekends as needed+1,"Houston, TX 77077",Just posted,23-07-14,"A Denny’s Host or Hostess has a “Guests First""...",https://www.indeed.com//rc/clk?jk=bcc00ac3cafb...
3,64e06c60e874df75938eec6f,TX14072023FRI1008,Server,"Full-time, Part-time",Denny's - 6131 Highway 6 S,NaN,Weekends as needed+1,"Missouri City, TX 77459",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=81be6634db19...
4,64e06c60e874df75938eec70,TX14072023FRI1009,Server,"Full-time, Part-time",Denny's - W. Pasadena Fwy,NaN,Weekends as needed+1,"Pasadena, TX 77506",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=5114425a2fc5...
...,...,...,...,...,...,...,...,...,...,...,...,...
145,64e06c60e874df75938eec40,NY14072023FRI3006,"Server - Brooklyn, NY",Full-time,Sushi Tokyo Inc,$20 - $45 an hour,NaN,"Brooklyn, NY 11223 (Gravesend-Sheepshead Bay a...",Today,23-07-14,Provide accurate orders to the kitchen staff. ...,https://www.indeed.com//rc/clk?jk=375a7aff9356...
146,64e06c60e874df75938eec41,NY14072023FRI3007,Food and Beverage Supervisor,Full-time,1 Hotel Brooklyn Bridge,$29 an hour,NaN,"Brooklyn, NY 11201",Today,23-07-14,That means hiring leaders for our Food & Bever...,https://www.indeed.com//rc/clk?jk=ee7acf1ff07e...
147,64e06c60e874df75938eec42,NY14072023FRI3008,Server Assistant (Earn up to $800/wk),Full-time,Au Cheval NYC,NaN,Weekends as needed+2,"New York, NY 11203 (East Flatbush area)",Today,23-07-14,Ofrecemos un conjunto completo de beneficios y...,https://www.indeed.com//rc/clk?jk=ffe1a20f22e0...
148,64e06c60e874df75938eec43,NY14072023FRI3009,Busperson - Conrad New York Downtown,NaN,Hilton,Estimated $39.4K - $49.8K a year,NaN,"New York, NY 10282 (Battery Park area)",Today,23-07-14,Respond to guest inquiries and requests and re...,https://www.indeed.com//rc/clk?jk=dd0d55f52262...


- Get Data
- Filter data
- set timer for posts
- Post at each group (based on location)

In [44]:
df.isna().sum()

_id              0
JobID            0
Title            0
Job_Type        12
Company          0
Salary          79
Schedule        73
Location         0
Date_Posted      0
Current_Date     0
Details          0
Apply_link       0
dtype: int64

In [45]:
df = df[df.Schedule.notnull()]
df

,_id,JobID,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Details,Apply_link
1,64e06c60e874df75938eec6c,TX14072023FRI1005,Host/Hostess,Part-time,Gyu-Kaku Japanese BBQ,NaN,Weekends as needed+2,"Houston, TX 77070 (Willowbrook area)",Just posted,23-07-14,Helpful attitude and friendly demeanor. He or ...,https://www.indeed.com//rc/clk?jk=9a6e26a88780...
2,64e06c60e874df75938eec6e,TX14072023FRI1007,Host,Part-time,Denny's - Westheimer Road,NaN,Weekends as needed+1,"Houston, TX 77077",Just posted,23-07-14,"A Denny’s Host or Hostess has a “Guests First""...",https://www.indeed.com//rc/clk?jk=bcc00ac3cafb...
3,64e06c60e874df75938eec6f,TX14072023FRI1008,Server,"Full-time, Part-time",Denny's - 6131 Highway 6 S,NaN,Weekends as needed+1,"Missouri City, TX 77459",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=81be6634db19...
4,64e06c60e874df75938eec70,TX14072023FRI1009,Server,"Full-time, Part-time",Denny's - W. Pasadena Fwy,NaN,Weekends as needed+1,"Pasadena, TX 77506",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=5114425a2fc5...
5,64e06c60e874df75938eec71,TX14072023FRI1010,Casual Dining Server,"Full-time, Part-time",TGI Friday's,NaN,Day shift+1,"Humble, TX 77338",Just posted,23-07-14,1 year server experience in a full-service res...,https://www.indeed.com//rc/clk?jk=31b7c78faaba...
...,...,...,...,...,...,...,...,...,...,...,...,...
140,64e06c60e874df75938eec3b,NY14072023FRI3001,Busser,"Full-time, Part-time",Lucciola Italian Restaurant,NaN,Day shift+6,"New York, NY 10024 (Upper West Side area)",Today,23-07-14,"Assist in setting up and clearing tables, ensu...",https://www.indeed.com//company/Piccolo-Cafe/j...
141,64e06c60e874df75938eec3d,NY14072023FRI3003,Part Time Server for FINE DINING Restaurant,"Full-time, Part-time",Lucciola Italian Restaurant,NaN,Monday to Friday+8,"New York, NY 10024 (Upper West Side area)",Today,23-07-14,Ability to thrive in a fast-paced and high-pre...,https://www.indeed.com//company/Piccolo-Cafe/j...
143,64e06c60e874df75938eec3e,NY14072023FRI3004,Host/Hostess,"Full-time, Part-time",DOMA Land + Sea,NaN,Monday to Friday+5,"Cedarhurst, NY 11516",Today,23-07-14,The host is expected to greet and seat our gue...,https://www.indeed.com//rc/clk?cmp=DOMA-Land-%...
147,64e06c60e874df75938eec42,NY14072023FRI3008,Server Assistant (Earn up to $800/wk),Full-time,Au Cheval NYC,NaN,Weekends as needed+2,"New York, NY 11203 (East Flatbush area)",Today,23-07-14,Ofrecemos un conjunto completo de beneficios y...,https://www.indeed.com//rc/clk?jk=ffe1a20f22e0...


In [46]:
df.isna().sum()

_id              0
JobID            0
Title            0
Job_Type         4
Company          0
Salary          73
Schedule         0
Location         0
Date_Posted      0
Current_Date     0
Details          0
Apply_link       0
dtype: int64

In [11]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [48]:
summarizer(df.Details[2], max_length = 150)[0]['summary_text']

Your max_length is set to 150, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


'a Denny’s Host or Hostess has a "Guests First" attitude . must be 16 or older .'

In [20]:
len(df)

71

In [11]:
summary[0]

{'summary_text': 'our servers go the extra mile to make it memorable . you’ll help with menu selections, quickly refill beverages and ensure food arrives on time, always delivering warm and friendly interactions . our busy restaurants mean consistent hours, flexible schedules and the opportunity to make a great income – as well as a lot of friends and memories along the way . if you have less experience, consider cashier or barista / Baker .'}

In [49]:
def message(df, i):
    return (
        f'**Test Post: {i}**\n'
        f'Post ID: {df.JobID[i]}\n'
        f'Job Opening: {df.Title[i]}\n'
        f'\n'
        f'{df.Location[i]}\n'
        f'\n'
        f'{df.Job_Type[i]}\n'
        f'\n'
        f'👉Comment "Interested" If You want to apply.\n'
        f'\n'
        f'Work Summary: \n'
        f"{df.Details[i]}\n"
        f'\n'
        f'#houstonjobs #houston #ergool #houstonjob #jobsearch #jobseekers #search #empower #career #nowhiring #jobopportunities #jobplacement #employmentassistance #jobupdates #restaurantjobs #retailjobs #jobopening #jobalert'
    )

In [40]:
count = 0
print(len(df))
for rows in range(len(df)):
    try:
        print(df['Details'][row])
    except:
        count += 1
        print(f"Key Error for row {row}")
        pass
print(count)

71
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row 1
Key Error for row

In [51]:
groups = ["674232410710265", "658314902622656"] #NY, TX
access_token = "EAAJHKZAZAZBPbsBO93UaRIKaegdz5ZCXNY47cWXYdeFa5NJZAZC9XSzF9sI1dNuKYQZANaUo6sxJaSHJyIvGgXwVAf3lRUovuushZB1xNkjtuqcFd9WGplYR4rG7ttsPJGnVx0ZCCGUMeoVuiGKcSvn6DWtLdEoXKZBIHzYjNghhirqjVo1zQ839UcjWnlRshSI9RQK0hTAIYFem8mjSyy74n5hGBZAAwROBTAUDdWmQ2lyQwW3REN1rZCoLW7LnZBdU7IwZDZD"
graph = facebook.GraphAPI(access_token=access_token)

for row in range(len(df)):
    print(f'Post No. {row}')
    #summary = summarizer(df.Details[row], max_length = 150)[0]['summary_text']
    if "NY" in df.Location[row]:
        try:
            x = graph.put_object(groups[0], 'feed', message=message(df, row))
            print(x)
            print("Posted Successfully in NY Group!")
        except:
            print("Error: Couldn't Post!")
    elif "TX" in df.Location[rows]:
        try:
            x = graph.put_object(groups[1], 'feed', message=message(df, row))
            print(x)
            print("Posted Successfully in the TX Group!")
        except:
            print("Error: Couldn't Post!")
    time.sleep(60)

Post No. 0


KeyError: 0

- change the summary
- Job Title
- RPM(REquest per minute) 200 requests per hour
- https://developers.facebook.com/docs/graph-api/overview/rate-limiting/
- 1 app multiple users
- 75 posts each day
- Unique ID's for the Jobs(e.g. NYTIME)
- Write an excel file from Digital Ocean (Unique_ID, Status, Apply_Link)